# Arbol Base 1ra Competencia

### 01) Carga de librerias y Datos

In [1]:
# Arbol elemental con libreria  rpart
# Debe tener instaladas las librerias  data.table  ,  rpart  y  rpart.plot

# cargo las librerias que necesito
require("data.table")
require("rpart")
require("rpart.plot") #Aqui se debe poner la ruta de la PC local

Loading required package: data.table

Loading required package: rpart

Loading required package: rpart.plot



In [2]:
# Aqui se debe poner la carpeta de la materia de SU computadora local (el Working Directory)
setwd("C:\\Users\\leandro.morinigo\\OneDrive - Accenture\\! DM en Econ y Fin 2023\\") 

#cargo el dataset
dataset <- fread("./datasets/competencia_01.csv") 

print(dim(dataset))

[1] 491063    155


In [3]:
# minima exploración por fecha
table(dataset$foto_mes)


202103 202104 202105 
163324 163637 164102 

In [4]:
# minima exploración por fecha y target
table(dataset$foto_mes, dataset$clase_ternaria )

        
                BAJA+1 BAJA+2 Continua
  202103      0   1049    963   161312
  202104   1363      0    963   161311
  202105   2790      0      0   161312

### 02) FEATURE ENGINEERING

In [5]:
#Reglas arbol simple sin 2 variables vinculadas: -ctrx_quarter -active_quarter
dataset[  , z1 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo<1257&mprestamos_personales<12840&ctarjeta_master<0.5) ]
dataset[  , z2 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo<1257&mprestamos_personales<12840&ctarjeta_master>=0.5) ]
dataset[  , z3 :=  as.integer(mcaja_ahorro>=343&mtarjeta_visa_consumo<3215&mpasivos_margen<183&cpayroll_trx<0.5) ]
dataset[  , z4 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo>=1257&cprestamos_personales<0.5&mautoservicio<699) ]

dataset[  , z5 :=  as.integer(mcaja_ahorro>=343&mpasivos_margen<183&cpayroll_trx>=0.5&mcomisiones_mantenimiento>=1770) ]
dataset[  , z6 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo<1257&mprestamos_personales>=12840&cdescubierto_preacordado<0.5) ]
dataset[  , z7 :=  as.integer(mcaja_ahorro>=343&mtarjeta_visa_consumo<1840&mpasivos_margen>=183&cpayroll_trx<0.5) ]
dataset[  , z8 :=  as.integer(mcaja_ahorro>=343&mtarjeta_visa_consumo>=3215&mpasivos_margen<183&cpayroll_trx<0.5) ]

dataset[  , z9 :=  as.integer (mcaja_ahorro>=343&mpasivos_margen>=183&cpayroll_trx>=0.5) ]
dataset[  , z10 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo<1257&mprestamos_personales>=12840&cdescubierto_preacordado>=0.5) ]
dataset[  , z11 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo>=1257&cprestamos_personales<0.5&mautoservicio>=699) ]
dataset[  , z12 :=  as.integer(mcaja_ahorro>=343&mtarjeta_visa_consumo>=1840&mpasivos_margen>=183&cpayroll_trx<0.5) ]


dataset[  , z13 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo>=1257&cprestamos_personales>=0.5&Visa_mconsumospesos>=12699) ]
dataset[  , z14 :=  as.integer(mcaja_ahorro>=343&mpasivos_margen<183&cpayroll_trx>=0.5&mcomisiones_mantenimiento<1770) ]
dataset[  , z15 :=  as.integer(mcaja_ahorro>=343&mpasivos_margen>=183&cpayroll_trx>=0.5&cliente_antiguedad>=3.5) ]
dataset[  , z16 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo>=1257&cprestamos_personales>=0.5&Visa_mconsumospesos<12699) ]

### 02) Modelado Rpart

In [6]:
#creo la clase_binaria SI={ BAJA+1, BAJA+2 }    NO={ CONTINUA }
dataset[ foto_mes==202103, 
         clase_binaria :=  ifelse( clase_ternaria=="Continua", "0L", "1L" ) ]

In [7]:
# SEPARO EN TRAIN(dtrain) Y TEST(dapply):
dtrain <- dataset[foto_mes == 202103] # defino donde voy a entrenar
dapply <- dataset[foto_mes == 202105] # defino donde voy a aplicar el model

print(paste("train:", dim(dtrain)[1], "| test:", dim(dapply)[1]))

[1] "train: 163324 | test: 164102"


**HIPERPARAMETROS** DEL MODELO

In [2]:
# genero el modelo,  aqui se construye el arbol
# quiero predecir clase_ternaria a partir de el resto de las variables
# Estos Hiperparametros NO se obtuvieron de una BO, los puse de base segun resultados de GridSearchs de las primeras clases
modelo  <- rpart(formula=   "clase_binaria ~ . -clase_ternaria",
                 data=      dtrain,  #los datos donde voy a entrenar
                 xval=         0,
                 cp=          -1,   # esto significa no limitar la complejidad de los splits
                 minsplit=  100,    # minima cantidad de registros para que se haga el split  
                 minbucket=  100,   # tamaño minimo de una hoja  
                 maxdepth=     12 ) # profundidad maxima del arbol

ERROR: Error in rpart(formula = "clase_binaria ~ . -clase_ternaria", data = dtrain, : no se pudo encontrar la función "rpart"


### 03) Prediccion

In [9]:
# aplico el modelo a los datos nuevos
prediccion <- predict(
        object = modelo,
        newdata = dapply,
        type = "prob"
)

In [10]:
# prediccion es una matriz con DOS columnas,
# cada columna es el vector de probabilidades
head(prediccion, n=2)

,0L,1L
1,0.9996777,0.0003222959
2,0.9985183,0.0014817086


In [11]:
# la ordeno de mayor a menor prob de baja+2
prediccion_df <- as.data.frame(prediccion)

head(prediccion_df[order(-prediccion_df$`1L`), ], n=3)

,0L,1L
,<dbl>,<dbl>
161,0.4666667,0.5333333
886,0.4666667,0.5333333
5507,0.4666667,0.5333333


In [12]:
# creo un df dfinal, que solo tiene el numero de cliente y la probabilidad para BAJA+2 = 1L
dfinal  <- copy( dapply[ , list(numero_de_cliente) ] )
dfinal[ , prob_SI := prediccion[ , "1L"] ]

head(dfinal ,n=3)

numero_de_cliente,prob_SI
<int>,<dbl>
29183981,0.0003222959
29184630,0.0014817086
29185433,0.0050722800


### 04) Cortes para Envios

In [13]:
# solo le envio estimulo a los registros con probabilidad de BAJA+2 mayor  a  1/40
set.seed(130631)  
dfinal[ , azar := runif( nrow(dapply) ) ]

# ordeno en forma descentente, y cuando coincide la probabilidad, al azar
setorder( dfinal, -prob_SI, azar )


head(dfinal ,n=7)

numero_de_cliente,prob_SI,azar
<int>,<dbl>,<dbl>
79631823,0.5333333,0.001370081
71876172,0.5333333,0.046091142
123229371,0.5333333,0.058287131
137412888,0.5333333,0.062144610
35810038,0.5333333,0.065692249
120481498,0.5333333,0.068384901
102356740,0.5333333,0.110064665


### 05) Exporto el archivo con las predicciones

In [14]:
# primero creo la carpeta donde va el experimento
dir.create("./exp/")
dir.create("./exp/KA4120")

Warning message in dir.create("./exp/"):
"'.\exp' already exists"
Warning message in dir.create("./exp/KA4120"):
"'.\exp\KA4120' already exists"


In [15]:
# genero el archivo para Kaggle -> SE SELECCIONO EL CORTE CON 9000 ENVIOS EN KAGGLE (COMPETENCIA 01)
for( corte  in  c( 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 10500, 11000 ) )
{
  #le envio a los  corte  mejores,  de mayor probabilidad de prob_SI
  dfinal[ , Predicted := 0L ]
  dfinal[ 1:corte , Predicted := 1L ]


  fwrite( dfinal[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
           file= paste0( "./exp/KA4120/KA4120_005_",  corte, ".csv"),
           sep=  "," )
}

### 06) Calcular el feature importance


In [16]:
# Calcular el feature importance
importancia <- modelo$variable.importance
importancia <- data.frame(Variable = names(importancia), Importancia = importancia)

head(importancia, n=40)

,Variable,Importancia
,<chr>,<dbl>
z1,z1,165.965663
z2,z2,70.736319
mrentabilidad,mrentabilidad,52.270763
mcomisiones,mcomisiones,44.093605
mcomisiones_otras,mcomisiones_otras,41.124170
mcomisiones_mantenimiento,mcomisiones_mantenimiento,40.871593
ctrx_quarter,ctrx_quarter,38.738434
mrentabilidad_annual,mrentabilidad_annual,37.802625
z3,z3,30.701569


In [17]:
fwrite(importancia,
        file = "./exp/KA4120/KA4120_005_FeImportance.csv",
        sep = ",")